In [1]:
import os
import shutil
import pickle
import numpy as np
import torch
import copy
import torch.nn as nn

from skopt.optimizer import gp_minimize
from skopt.space import Real, Categorical, Integer

from bcnf.simulation.physics import get_data
from bcnf.models.cnf import CondRealNVP
from bcnf.models.feature_network import FullyConnectedFeatureNetwork
from bcnf.eval.crossvalidate import cross_validate
from bcnf.errors import TrainingDivergedError

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
np.int = np.int64

In [3]:
# Check if a checkpoint exists
checkpoint_file = 'checkpoint.pkl'

In [4]:
X, y = get_data(
    T=5.0,
    dt=0.1,
    N=2_500,
    break_on_impact=False
)

X_tensor = torch.Tensor(X.reshape(X.shape[0], -1))
y_tensor = torch.Tensor(y)

 48%|████▊     | 1203/2500 [00:02<00:02, 590.68it/s]


KeyboardInterrupt: 

In [ ]:
optimizer_kwargs = {
    "lr": 1e-3
}

lr_scheduler_kwargs = {
    "mode": "min",
    "factor": 0.5,
    "patience": 50,
}

In [ ]:
# Example search spaces
search_spaces = {
    'condition_size': Integer(1, 2048),
    'model_hidden_size': Integer(32, 256),
    'model_n_blocks': Integer(1, 64),
    'model_act_norm': Categorical([True, False]),
    'feature_network_hidden_size': Integer(16, 2048),
    'feature_network_hidden_layers': Integer(0, 16),
    'feature_network_dropout': Real(0, 0.5),
}

In [ ]:
def param_index(name, search_spaces):
    return list(search_spaces.keys()).index(name)

In [ ]:
model_size = y_tensor.shape[1]
feature_size = X_tensor.shape[1]

In [ ]:
def score_parameters(params: list):
    try:
        fold_metrics = cross_validate(
            model_class=CondRealNVP,
            model_kwargs={
                "size": model_size,
                "hidden_size": params[param_index('model_hidden_size', search_spaces)],
                "n_blocks": params[param_index('model_n_blocks', search_spaces)],
                "n_conditions": params[param_index('condition_size', search_spaces)],
                "act_norm": params[param_index('model_act_norm', search_spaces)],
            },
            feature_network_class=FullyConnectedFeatureNetwork,
            feature_network_kwargs={
                "sizes": [feature_size]
                    + [params[param_index('feature_network_hidden_size', search_spaces)]] * params[param_index('feature_network_hidden_layers', search_spaces)]
                    + [params[param_index('condition_size', search_spaces)]],
                "dropout": params[param_index('feature_network_dropout', search_spaces)],    
            },
            optimizer_class=torch.optim.Adam,
            optimizer_kwargs=optimizer_kwargs,
            lr_scheduler_class=torch.optim.lr_scheduler.ReduceLROnPlateau,
            lr_scheduler_kwargs=lr_scheduler_kwargs,
            X=X_tensor,
            y=y_tensor,
            n_epochs=50_000,
            val_loss_patience=200,
            val_loss_tolerance=1e-3,
            batch_size=256,
            device=device,
            verbose=True,
            n_splits=3,
            errors="raise"
        )
    except TrainingDivergedError as e:
        print(e)
        return 100  # A big number

    val_loss_list = [r['val_loss'] for r in fold_metrics]
    return np.mean(val_loss_list) + np.std(val_loss_list)

In [ ]:
# Define a callback function to save the results
def save_checkpoint(result):
    # Save the checkpoint after each iteration or at the end of the optimization
    with open(checkpoint_file + ".tmp", 'wb') as f:
        # Ignore
        # - result['specs']['args']['func']
        # - result['specs']['args']['callback']
        # because it causes problems when reading somewhere else
        result_no_func = copy.deepcopy(result)
        del result_no_func['specs']['args']['func']
        del result_no_func['specs']['args']['callback']
        pickle.dump(result_no_func, f)

    # Delete the old checkpoint file and rename the temporary file
    shutil.move(checkpoint_file + ".tmp", checkpoint_file)

In [ ]:
N_STEPS = 100

In [ ]:
if os.path.exists(checkpoint_file):
    print(f'Loading checkpoint from {checkpoint_file}')
    with open(checkpoint_file, 'rb') as f:
        checkpoint = pickle.load(f)
        checkpoint['specs']['args']['func'] = score_parameters
        checkpoint['specs']['args']['callback'] = save_checkpoint
    print(f'Resuming from iteration {len(checkpoint.x_iters)}')
else:
    print('No checkpoint found. Starting new optimization')
    checkpoint = None

# You might want to adjust the n_calls or other parameters based on the checkpoint
result = gp_minimize(
    func=score_parameters,
    dimensions=search_spaces.values(),
    n_initial_points=10,
    n_calls=N_STEPS - len(checkpoint.x_iters) if checkpoint else N_STEPS,
    random_state=42,
    verbose=True,
    callback=save_checkpoint,
    x0 = checkpoint.x_iters if checkpoint else None,
    y0 = checkpoint.func_vals if checkpoint else None)

No checkpoint found. Starting new optimization
Iteration No: 1 started. Evaluating function at random point.


  0%|          | 0/50000 [00:00<?, ?it/s]


Error in fold 0: Loss exploded to 172962.921875 at epoch 0.42857142857142855
Loss exploded to 172962.921875 at epoch 0.42857142857142855
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.7971
Function value obtained: 100.0000
Current minimum: 100.0000
Iteration No: 2 started. Evaluating function at random point.


Train: 15.4042 - Val: 9.9759 (avg: 20.4454, min: 20.9964) | lr: 1.00e-03 - Patience: 1/200:   0%|          | 2/50000 [00:00<3:54:20,  3.56it/s]


Error in fold 0: Loss exploded to 6670967296.0 at epoch 2.4285714285714284
Loss exploded to 6670967296.0 at epoch 2.4285714285714284


KeyError: 'func'

In [ ]:
if os.path.exists(checkpoint_file):
    print(f'Loading checkpoint from {checkpoint_file}')
    with open(checkpoint_file, 'rb') as f:
        checkpoint = pickle.load(f)

Loading checkpoint from checkpoint.pkl


In [ ]:
checkpoint['specs']['args']['callback']

<function __main__.save_checkpoint(result)>